In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
    
test_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')

print(len(dev_users_list))
print(len(test_users_list))

3000
['#d6866a498157771069fdf15361cb012b', '#f963fb8c5d9d14d503fc4e80bd8617b4', '#87a6479c91e4276374378f1d28eb307c', '#677e984e245b344f61dc5d3cc1f352c8', '#519f45eb14e4807e8714fb7e835463eb']
5000
['#7ee14df8642a7925b1465ff5c89efe5b', '#8420b9385b282028eebf1ad6b4a221c0', '#c9b31d8b64357f5854b1ba55b32eb6d3', '#9bb1e13b5481fa3737af20870b25c723', '#37d5f99a7f12c9ba90c4e2ac92e54ab6']


In [2]:
recommend_new = []
with open(directory + 'recommend.txt') as f:
    while True:
        line = f.readline()
        if not line: break
        recommend_new.append(line)
recommend_new_top_100 = recommend_new[0][33:-1]

In [3]:
with open(directory + 'article_by_testuser.json') as f:
    article_seen_by_testuser = json.load(f)

In [4]:
rntl = recommend_new_top_100.split(' ')[1:]

In [5]:
with open(directory + 'rasby_devuser.csv',encoding='utf-8') as f:
    r1 = pd.read_csv(f)

In [14]:
r1 = r1.drop('Unnamed: 0',1)
r1['read_dt']  = r1['read_dt'].map(lambda x : x.replace('-',''))

In [20]:
a = list(r1['class'][:1])[0]
a

'5%'

In [21]:
def int_class(x):
    if x == '5%':
        result = 5
    elif x == '10%':
        result = 10
    elif x == '25%':
        result = 25
    elif x == '50%':
        result = 50
    elif x == '75%':
        result = 75
    else:
        result = 100
    return result

In [22]:
# 최신에 읽은 글들 중 20190201 이후에 읽은 글들만 모은다
a = '20190201'
r2 = r1[r1['read_dt'] >= a]

In [25]:
import time
from tqdm import tqdm

In [39]:
# 읽은 구독자 list 도 만들기
# 개인 or magazine

author_lists = []
class_lists = []
magazine_lists = []
unique_author = []
for i in tqdm(range(3000),mininterval = 1):
    recent_read_raw = r2[r2['user_id'] == dev_users_list[i]]       
    len_recent_read_raw = len(recent_read_raw)
    
    # class mean
    class_sum = 0
    magazine_sum = 0
    # 구독자 정보 모으기
    author_list = []
    author_cnt = []
    for j in range(len_recent_read_raw):
        # class mean
        rrr = recent_read_raw[j:j+1]
        class_int = int_class(list(rrr['class'])[0])
        class_sum += class_int
        # magainze mean
        magazine_type = list(rrr['type'])[0]
        if(magazine_type == '매거진'):
            magazine_sum += 1
        # author list
        author = list(rrr['author_id'])[0]
        found = 0
        for k in range(len(author_list)):
            if author_list[k] == author:
                author_cnt[k] += 1
                found = 1
                break
        if(found == 0):
            author_list.append(author)
            author_cnt.append(1)

    author_list_dict = dict()
    for j in range(len(author_list)):
        author_list_dict[author_list[j]] = author_cnt[j]

    if(len_recent_read_raw != 0 ):
        class_mean = class_sum / len_recent_read_raw
        class_lists.append(class_mean)
        
        magazine_mean = magazine_sum / len_recent_read_raw
        magazine_lists.append(magazine_mean)
        
        author_list_dict = sorted(author_list_dict.items(), key=(lambda x: x[1]), reverse = True) # 이러면 [ (), () ]형태
        author_lists.append(author_list_dict)
        
        unique_author.append(len(author_list))
    else:
        class_lists.append(5.0)
        magazine_lists.append(1.0)
        author_lists.append([])
        unique_author.append(0)



100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:26<00:00, 113.98it/s]


r2 = 2019 2월 한달간 유저가 읽은 max(20)개 글들의 정보들이 dev_users_list의 인덱스 순서로 저장
    -> 평균 = 16.29 개의 글이 있음 

dev_users_list의 index 별로 해당 정보들이 저장되어 있음

author_lists : (읽은 작가) : cnt, cnt기준 정렬
    -> ex) author_lists[0] : [('@tenbody', 12), ('@forchoon', 2), ('@gakugo', 1), ('@barugi', 1)]
    -> ex) author_lists[1] : [('@dailylife', 2), ('@k52524', 1)]
    
unique_author = 소비 글들의 고유한 작가 수 
    -> ex) unique_author[0] : 11 
    -> 평균 = 7.89 = sum(unique_author) / 3000
    
class_lists = 소비 글의 class 평균 
    -> ex) class_lists[0] : 5.5 
    -> 평균 = 11.92 = sum(class_lists) / 3000 
    
magazine_lists = 소비 글들의 magazine 비율 
    -> ex) magazine_lists[0] : 0.85
    -> 평균 = 0.76

In [96]:
# 이상하게 저장된 글
r1[11679:11680]

,read_dt,user_id,article_id,title,sub_title,author_id,reg_dt,type,display_url,keyword_list,magazine_id,off_day,read_cnt,class
11679,"너와 있는 시간을 사고 싶다, 시라쿠사의 바다 (1)\n",@nimathebride,2019-01-11,매거진,https://brunch.co.kr/@nimathebride/102,"['시칠리아', '이탈리아', '유럽여행']",36355,0,83,10%,NaN,NaN,NaN,NaN
